In [1]:
# import the necessary packages
from imutils.video import VideoStream
import numpy as np
import argparse
import imutils
import time
import cv2
import os

protextFile = 'deploy.prototxt'
caffeFile = 'res10_300x300_ssd_iter_140000.caffemodel'

video1='face-demographics-walking.mp4'
video2='head-pose-face-detection-female.mp4'
video3='head-pose-face-detection-female-male.mp4'

folderName = 'DetectedFaces'

In [2]:
net = cv2.dnn.readNetFromCaffe(protextFile,caffeFile)


In [3]:
count = 0
vidcap = cv2.VideoCapture(video3)

# loop over the frames from the video stream
while (vidcap.isOpened()):
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 400 pixels
    success,frame = vidcap.read(0)
    if success:
        frame = imutils.resize(frame, width=400)
 
        # grab the frame dimensions and convert it to a blob
        (h, w) = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,(300, 300), (104.0, 177.0, 123.0))
 
        # pass the blob through the network and obtain the detections and
        # predictions
        net.setInput(blob)
        detections = net.forward()
    
        # loop over the detections
        for i in range(0, detections.shape[2]):
            # extract the confidence (i.e., probability) associated with the
            # prediction
            confidence = detections[0, 0, i, 2]
        
            # filter out weak detections by ensuring the `confidence` is
            # greater than the minimum confidence
            if (confidence *100) < 50.0:
                continue

            # compute the (x, y)-coordinates of the bounding box for the
            # object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            
            # draw the bounding box of the face along with the associated
            # probability
            text = "{:.2f}%".format(confidence * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(frame, (startX, startY), (endX, endY),(0, 0, 255), 2)
            cv2.putText(frame, text, (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
            
            face_roi=frame[startY:endY,startX:endX]
            filename = "face{}_{}.jpg".format(os.getpid(),count) 
            #Create folder
            if not os.path.exists(folderName):
                os.mkdir(folderName)
                print("Directory " , folderName ,  " Created ")
            
            cv2.imwrite(folderName+'\\'+filename,frame)
            count += 1
        # show the output frame
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF
 
        # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            break
        success,image = vidcap.read()
    else:
        print('No Image/Video')
        break
# do a bit of cleanup
vidcap.release()
cv2.destroyAllWindows()

Directory  DetectedFaces  Created 
